In [8]:
from astropy.io import fits
import pandas as pd
import os

# Dizin içindeki tüm FITS dosyalarını al
fits_directory = '/Users/nurullah/Desktop/true_PNe/son_alfa_out/neat_out/adsız/'
fits_file_paths = [os.path.join(fits_directory, file) for file in sorted(os.listdir(fits_directory)) if file.endswith('.fits')]

# Excel dosyasının tam yolu ve ismi
excel_file_path = '/Users/nurullah/Desktop/true_PNe/son_alfa_out/Bde.xlsx'

# Her FITS dosyası için döngü
with pd.ExcelWriter(excel_file_path, engine='openpyxl') as writer:
    for fits_file_path in fits_file_paths:
        # Open the FITS file
        with fits.open(fits_file_path) as hdul:
            # Access the data and header of the LINES BinTableHDU (hdul[2])
            data = hdul[2].data

            # Convert the FITS data to a pandas DataFrame
            df = pd.DataFrame(data)

            # Sadece belirtilen sütunları seç
            selected_columns = ['WlenRest', 'Flux', 'Uncertainty', 'Ion', 'DereddenedFlux', 'DereddenedFluxLo', 'DereddenedFluxHi']
            df_selected = df[selected_columns]

            # Negatif ve sıfıra eşit olan değerlere sahip satırları filtrele ve sil
            df_selected = df_selected[df_selected['Flux'] > 0.01]

            # Sayfa adını dosya isminin ilk altı karakterine in- ekini ekleyerek belirle
            sheet_name = f'{os.path.basename(fits_file_path)[:6]}'

            # Verileri içeren DataFrame'i yeni bir sayfada kaydet
            df_selected.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Data has been successfully exported to {excel_file_path}")

Data has been successfully exported to /Users/nurullah/Desktop/true_PNe/son_alfa_out/neat_out/Bde.xlsx


In [20]:
import pandas as pd
import os

# Excel dosyasının tam yolu ve ismi
excel_file_path = '/Users/nurullah/Desktop/true_PNe/son_alfa_out/neat_out/hdul1.xlsx'

# Excel dosyasını oku
df = pd.read_excel(excel_file_path, sheet_name=None)

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her sayfa için döngü
for sheet_name, sheet_data in df.items():
    # Eğer ana DataFrame boşsa, direkt ekle
    if main_df.empty:
        main_df = sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']].copy()
        main_df.rename(columns={'Flux': f'Flux_{sheet_name}', 'Uncertainty': f'Uncertainty_{sheet_name}'}, inplace=True)
    else:
        # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
        main_df = main_df.merge(sheet_data[['WlenRest', 'Ion', 'Flux', 'Uncertainty']],
                                on=['WlenRest', 'Ion'],
                                how='outer',
                                suffixes=('', f'_{sheet_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'Flux_Tablo'

# Ana DataFrame'i Excel dosyasına ekleyerek belirtilen sayfa adını kullan
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
    if new_sheet_name in writer.sheets:
        # Sayfa zaten varsa, üzerine yazabilir veya farklı bir isimle kaydedebilirsiniz.
        raise ValueError(f"Sheet '{new_sheet_name}' already exists. Choose a different sheet name.")
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")

Data has been successfully exported to /Users/nurullah/Desktop/true_PNe/Çıktı.xlsx


In [ ]:
import pandas as pd
import os

# Excel dosyasının tam yolu ve ismi
excel_file_path = '/Users/nurullah/Desktop/true_PNe/son_alfa_out/neat_out/hdul1.xlsx'
# Excel dosyasını oku
df = pd.read_excel(excel_file_path, sheet_name=None)

# Birleştirilecek verileri tutacak ana DataFrame'i oluştur
main_df = pd.DataFrame()

# Her sayfa için döngü
for sheet_name, sheet_data in df.items():
    # Sayfadaki sütunları kontrol et
    required_columns = ['WlenRest', 'Ion', 'DereddenedFlux', 'DereddenedFluxLo', 'DereddenedFluxHi']
    if not all(column in sheet_data.columns for column in required_columns):
        print(f"Not all required columns {required_columns} are present in sheet {sheet_name}. Skipping this sheet.")
        continue

    # Eğer ana DataFrame boşsa, direkt ekle
    if main_df.empty:
        main_df = sheet_data[required_columns].copy()
        main_df.rename(columns={'DereddenedFlux': f'DereddenedFlux_{sheet_name}',
                                'DereddenedFluxLo': f'DereddenedFluxLo_{sheet_name}',
                                'DereddenedFluxHi': f'DereddenedFluxHi_{sheet_name}'}, inplace=True)
    else:
        # Ana DataFrame'de 'WlenRest' ve 'Ion' sütunlarına göre birleştirme yap
        main_df = main_df.merge(sheet_data[required_columns],
                                on=['WlenRest', 'Ion'],
                                how='outer',
                                suffixes=('', f'_{sheet_name}'))

# NaN değerleri '-' ile değiştir
main_df.fillna('-', inplace=True)

# DereddenedFluxLo ve DereddenedFluxHi değerlerini hesapla
for sheet_name in df.keys():
    # Sütunun var olup olmadığını kontrol et
    if f'DereddenedFluxLo_{sheet_name}' in main_df.columns and f'DereddenedFluxHi_{sheet_name}' in main_df.columns:
        # Dönüşüm yapmadan önce '-' değerlerini NaN olarak değiştir
        main_df[f'DereddenedFluxLo_{sheet_name}'].replace('-', pd.NA, inplace=True)
        main_df[f'DereddenedFluxHi_{sheet_name}'].replace('-', pd.NA, inplace=True)

        # Sayısal değerlere dönüştür
        main_df[f'DereddenedFluxLo_{sheet_name}'] = pd.to_numeric(main_df[f'DereddenedFluxLo_{sheet_name}'], errors='coerce')
        main_df[f'DereddenedFluxHi_{sheet_name}'] = pd.to_numeric(main_df[f'DereddenedFluxHi_{sheet_name}'], errors='coerce')

        # DereddenedFluxLo ve DereddenedFluxHi değerlerini hesapla
        main_df[f'DereddenedFluxLo_{sheet_name}'] = main_df[f'DereddenedFluxLo_{sheet_name}'] - main_df[f'DereddenedFlux_{sheet_name}']
        main_df[f'DereddenedFluxHi_{sheet_name}'] = main_df[f'DereddenedFluxHi_{sheet_name}'] - main_df[f'DereddenedFlux_{sheet_name}']
    else:
        print(f"Warning: 'DereddenedFluxLo_{sheet_name}' or 'DereddenedFluxHi_{sheet_name}' not found in the DataFrame. Skipping this sheet.")

# 'WlenRest' sütununa göre küçükten büyüğe sırala
main_df.sort_values('WlenRest', inplace=True)

# Excel dosyasına ekleyeceğimiz sayfa adı
new_sheet_name = 'DereddenedFlux_latex'

# Ana DataFrame'i Excel dosyasına ekleyerek belirtilen sayfa adını kullan
with pd.ExcelWriter(excel_file_path, engine='openpyxl', mode='a') as writer:
    if new_sheet_name in writer.sheets:
        # Sayfa zaten varsa, üzerine yazabilir veya farklı bir isimle kaydedebilirsiniz.
        raise ValueError(f"Sheet '{new_sheet_name}' already exists. Choose a different sheet name.")
    main_df.to_excel(writer, sheet_name=new_sheet_name, index=False)

print(f"Merged, sorted data has been successfully added to the sheet {new_sheet_name} in the Excel file.")